In [49]:
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [50]:
import sklearn
sklearn.__version__

'1.0.1'

In [51]:
#import comet_ml in the top of your file
from comet_ml import Experiment
import os
import matplotlib.pyplot as plt
import joblib
from copy import deepcopy
import numpy as np
from sklearn.feature_selection import RFECV
import json
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.preprocessing import StandardScaler
from skopt import BayesSearchCV
from sklearn import feature_selection
from sklearn.metrics import f1_score, precision_score, recall_score, confusion_matrix
import xgboost as xgb
import pandas as pd
from ift6758.features.feature_engineering2 import SeasonDataSetTwo
from ift6758.metrics import plot_metrics
import random
random_state = 42
import tempfile


In [60]:
years = [2019]
dataset = SeasonDataSetTwo(years)
df_total = dataset.get_features_2()
df = df_total


File already Exists, loading from ../ift6758/data/PICKLE//2019_features2.pkl


In [61]:
X = df[["shotDistance","shotAngle"]].iloc[:5]
X

,shotDistance,shotAngle
0,4.123106,14.036243
1,57.035077,-2.009554
2,26.683328,12.994617
3,36.055513,-33.690068
4,55.226805,-31.675469


In [62]:
import requests
import json
r = requests.post(
	"http://127.0.0.1:5000/predict", 
	json=json.loads(X.to_json())
)
print(r.json())



{"probaIsGoal":{"0":0.1542563787,"1":0.0567860263,"2":0.1020496683,"3":0.0856855542,"4":0.0589828114},"predictionIsGoal":{"0":0,"1":0,"2":0,"3":0,"4":0}}


In [63]:
import json
import requests
import pandas as pd
from ift6758.ift6758.client.serving_client import ServingClient

In [64]:
sc = ServingClient()


In [65]:
sc.predict(X)

,probaIsGoal,predictionIsGoal
0,0.154256,0
1,0.056786,0
2,0.102050,0
3,0.085686,0
4,0.058983,0


In [66]:
sc.logs()

"INFO:app:Welcome to the work of Team 11\nINFO:app:Initialization\nINFO:app:Default Classifier Loaded logreg-distance-angle\nINFO:app:{'shotDistance': {'0': 4.1231056256, '1': 57.0350769264, '2': 26.6833281283, '3': 36.0555127546, '4': 55.2268050859}, 'shotAngle': {'0': 14.0362434679, '1': -2.009553813, '2': 12.9946167919, '3': -33.690067526, '4': -31.6754687381}}\nINFO:app:{\"probaIsGoal\":{\"0\":0.1542563787,\"1\":0.0567860263,\"2\":0.1020496683,\"3\":0.0856855542,\"4\":0.0589828114},\"predictionIsGoal\":{\"0\":0,\"1\":0,\"2\":0,\"3\":0,\"4\":0}}\nINFO:werkzeug:127.0.0.1 - - [16/Dec/2021 23:21:33] \"POST /predict HTTP/1.1\" 200 -\nINFO:app:{'shotDistance': {'0': 4.1231056256, '1': 57.0350769264, '2': 26.6833281283, '3': 36.0555127546, '4': 55.2268050859}, 'shotAngle': {'0': 14.0362434679, '1': -2.009553813, '2': 12.9946167919, '3': -33.690067526, '4': -31.6754687381}}\nINFO:app:{\"probaIsGoal\":{\"0\":0.1542563787,\"1\":0.0567860263,\"2\":0.1020496683,\"3\":0.0856855542,\"4\":0.05898

'"INFO:app:Welcome to the work of Team 11\\nINFO:app:Initialization\\nINFO:app:Default Classifier Loaded logreg-distance-angle\\nINFO:app:{\'shotDistance\': {\'0\': 4.1231056256, \'1\': 57.0350769264, \'2\': 26.6833281283, \'3\': 36.0555127546, \'4\': 55.2268050859}, \'shotAngle\': {\'0\': 14.0362434679, \'1\': -2.009553813, \'2\': 12.9946167919, \'3\': -33.690067526, \'4\': -31.6754687381}}\\nINFO:app:{\\"probaIsGoal\\":{\\"0\\":0.1542563787,\\"1\\":0.0567860263,\\"2\\":0.1020496683,\\"3\\":0.0856855542,\\"4\\":0.0589828114},\\"predictionIsGoal\\":{\\"0\\":0,\\"1\\":0,\\"2\\":0,\\"3\\":0,\\"4\\":0}}\\nINFO:werkzeug:127.0.0.1 - - [16/Dec/2021 23:21:33] \\"POST /predict HTTP/1.1\\" 200 -\\nINFO:app:{\'shotDistance\': {\'0\': 4.1231056256, \'1\': 57.0350769264, \'2\': 26.6833281283, \'3\': 36.0555127546, \'4\': 55.2268050859}, \'shotAngle\': {\'0\': 14.0362434679, \'1\': -2.009553813, \'2\': 12.9946167919, \'3\': -33.690067526, \'4\': -31.6754687381}}\\nINFO:app:{\\"probaIsGoal\\":{\\"0\

In [67]:
r=sc.download_registry_model(workspace="morph-e",model="xgbase-bayestuning",version="1.0.0")


In [68]:
r.json()

{'new_classifier': 'xgbase-bayestuning'}